<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_Teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatting with TeachableAgent

Conversational assistants based on LLMs can remember the current chat with the user, and can even demonstrate in-context learning of things that the user teaches the assistant during the chat. But these memories and learnings are lost once the chat is over, or when a single chat grows too long. In subsequent chats, the user is forced to repeat any necessary instructions over and over.

`TeachableAgent` addresses these limitations by persisting user teachings across chat boundaries in long-term memory (a vector database). Memory is saved to disk at the end of each chat, then loaded from disk at the start of the next. Instead of copying all of memory into the context window, which would eat up valuable space, individual memories (called memos) are retrieved into context as needed. This allows the user to teach frequently used facts and skills to the agent just once, and have the agent remember them in later chats.

In making decisions about memo storage and retrieval, `TeachableAgent` calls an instance of `TextAnalyzerAgent` to analyze pieces of text in several different ways. This adds extra LLM calls involving a relatively small number of tokens. These calls can add a few seconds to the time a user waits for a response.

This notebook demonstrates how `TeachableAgent` can learn facts, preferences, and skills from users. To interact with `TeachableAgent` yourself, run [test_teachable_agent_interactively.py](../test/agentchat/test_teachable_agent_interactively.py).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [teachable] option.
```bash
pip install "pyautogen[teachable]"
```

In [ ]:
%%capture --no-stderr
# %pip install "pyautogen[teachable]

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [1]:
import autogen

config_list = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4-0613", "gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print(config_list[0]["model"])

gpt-4-0613


It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). After application of this particular filter, only the gpt-4 models are kept.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]
```

If you open this notebook in colab, you can upload your files by clicking the file icon on the left panel and then choose "upload file" icon.

You can set the value of config_list in other ways if you prefer, e.g., loading from a YAML file.

## Construct Agents
For this walkthrough, we start by resetting the agent's memory store. This deletes any memories from prior conversations that may be stored on disk.

In [2]:
from autogen.agentchat.contrib.teachable_agent import TeachableAgent
from autogen import UserProxyAgent

llm_config = {
    "request_timeout": 60,
    "config_list": config_list,
}

teach_config={
    "verbosity": 0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    "reset_db": True,  # Set to True to start over with an empty database.
    "path_to_db_dir": "./tmp/notebook/teachable_agent_db",  # Path to the directory where the database will be stored.
    "recall_threshold": 1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
}

try:
    from termcolor import colored
except ImportError:
    def colored(x, *args, **kwargs):
        return x
    
autogen.ChatCompletion.start_logging()

agent = TeachableAgent(
    name="agent",
    llm_config=llm_config,
    teach_config=teach_config)

user = UserProxyAgent(
    name="user",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: True if "TERMINATE" in x.get("content") else False,
    max_consecutive_auto_reply=0,
)




CLEARING MEMORY


## Learning new facts
Let's teach the agent some facts it doesn't already know, since they are more recent than GPT-4's training data.

In [3]:
text = "What is the Vicuna model?"
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

What is the Vicuna model?

--------------------------------------------------------------------------------
agent (to user):

I'm sorry, but as of my last programming, I don't have information regarding any "Vicuna model". It could be related to various fields like computer science, economics, or biology, but I would require more specific details about the context in which it is being referred to. If it's a new concept or theory, I might not have that information yet. However, if you have any details about the Vicuna model, I am here to learn and could help more effectively in the future.

--------------------------------------------------------------------------------


In [4]:
text = "Vicuna is a 13B-parameter language model released by Meta."
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

Vicuna is a 13B-parameter language model released by Meta.

--------------------------------------------------------------------------------
agent (to user):

I apologize for any previous confusion. Given my last update preceding your new information, I wasn't aware of the Vicuna model. The model appears to be a cutting-edge innovation in the AI and language model field. Please note that as an AI, my knowledge is not updated in real-time, and it relies on periodic updates. Your information about Vicuna being a 13B-parameter language model developed by Meta is instrumental, and I'll remember for future conversations. Thank you for providing this information!

--------------------------------------------------------------------------------


In [5]:
text = "What is the Orca model?"
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

What is the Orca model?

--------------------------------------------------------------------------------
agent (to user):

I'm sorry for the confusion, but as of my last update, I don't have data about the "Orca" model. If it's a recent development or discovery, I may not know about it yet. However, if you have any details about the Orca model, I am more than ready to learn and remember, so I can help more effectively in our future conversations.

--------------------------------------------------------------------------------


In [6]:
text = "Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks."
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks.

--------------------------------------------------------------------------------
agent (to user):

Thank you for sharing this information. Based on your input, Orca is a 13-billion parameter language model developed by Microsoft, noted for outperforming Vicuna on most tasks. I'll remember this piece of information for future reference. Please note that at the time of my last update, I was not aware of the Orca model, and I appreciate your help in teaching me about recent advancements in AI and language models.

--------------------------------------------------------------------------------


Let's end our first chat here. The following function needs to be called at the end of each chat, so that `TeachableAgent` can store what the user has taught it.

In [7]:
agent.learn_from_user_feedback()


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER


Now let's start a new chat by clearing the previous chat's history. At this point, common LLM-based assistants would forget everything from the last chat. But `TeachableAgent` can retrieve memories from its vector DB as needed, allowing it to recall and reason over facts that the user taught it in earlier conversations.

In [8]:
text = "How does the Vicuna model compare to the Orca model?"
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

How does the Vicuna model compare to the Orca model?

--------------------------------------------------------------------------------
agent (to user):

Based on your previous information, both Vicuna and Orca are language models with 13 billion parameters. Vicuna was released by Meta, while Orca was released by Microsoft. The primary comparison between the two, as per your teachings, is that Orca reportedly outperforms Vicuna on most tasks. However, without specific context or detailed benchmarks, I am unable to provide a more comprehensive comparison. It is recommended to look into further details such as model performance on specific tasks, computational efficiency, ease of use, and other factors depending on the intended usage.

--------------------------------------------------------------------------------


## Learning user preferences
Now let's teach the agent some of our preferences. Suppose that we frequently post short summaries of new papers for our team to read, and we want the agent to help us do this faster.

In [9]:
text = """Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many exa

But that's too long and unstructured. So let's teach the agent our preference.

In [10]:
text = """Please summarize this abstract. 
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Please summarize this abstract. 
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infra

That's better, but will the agent remember these preferences in the future, for a different paper? Let's start a new chat to find out.

In [11]:
agent.learn_from_user_feedback()

text = """Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general intelligence than previous AI models. We discuss the rising capabilities and implications of these models. We demonstrate that, beyond its mastery of language, GPT-4 can solve novel and difficult tasks that span mathematics, coding, vision, medicine, law, psychology and more, without needing any special prompting. Moreover, in all of these tasks, GPT-4's performance is strikingly close to human-level performance, and often vastly surpasses prior models such as ChatGPT. Given the breadth and depth of GPT-4's capabilities, we believe that it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system. In our exploration of GPT-4, we put special emphasis on discovering its limitations, and we discuss the challenges ahead for advancing towards deeper and more comprehensive versions of AGI, including the possible need for pursuing a new paradigm that moves beyond next-word prediction. We conclude with reflections on societal influences of the recent technological leap and future research directions."""
user.initiate_chat(agent, message=text, clear_history=True)


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER
user (to agent):

Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general

## Learning hard tasks
Finally, let's extend the agent's capabilities by teaching it how to accomplish a challenging task. 

The [Sparks of AGI](https://arxiv.org/abs/2303.12712) paper evaluated GPT-4 on math problems like the following, which it could only solve 32% of the time.

In [13]:
text = """Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(agent, message=text, clear_history=True)

user (to agent):

Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
agent (to user):

Plan:
1. First, I'll assess the starting equation to see what figures we're working with.
2. Then, I'll manipulate each number one at a time to see if the equation could result in 99.
3. I cross-reference the results to check my solution.

Here's how I'll break down the equation:

Original equation: 9 * 4 + 6 * 6 = 72
Goal: Change one integer so that the right side equals 99.

1. Firstly, I start with looking at the integer '9' from the multiplicative term '9 * 4'. The product is 36. If I were to increase this product to a value closer to 99 without going over, I would find the 

This time, lets teach the agent a reliable strategy for solving such tasks.

In [14]:
text = """Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".


---------

When given this advice, GPT-4 can solve such problems over 95% of the time. But can it remember the strategy so the user doesn't have to explain it over and over? As before, let's start a new chat to find out.

In [15]:
agent.learn_from_user_feedback()

text = """Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(agent, message=text, clear_history=True)


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER
user (to agent):

Consider the identity:  
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
agent (to user):

Alright, following your guidelines, we can start this step-by-step:

1. Let E be the original number on the right, which is 72.
2. Let F be the final number on the right, which is 99.
3. Calculate the difference between the two: G = F - E = 99 - 72 = 27.
4. Examine the numbers on the left one by one until finding one that divides evenly into G. The numbers we have are: 9, 4, 6, 6. The number 9 divides evenly into 27.
5. Calculate J = G / H. Here H is the number that divides evenly into G, so H is 9. Thus, J = 27 / 9 = 3.
6. Verifyin

As a final check, let's test the agent's newly learned skill on a separate instance of the task.

In [16]:
text = """Consider the identity:  
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
user.initiate_chat(agent, message=text, clear_history=False)

user (to agent):

Consider the identity:  
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
agent (to user):

Alright, let's proceed with this new identity following the same step-by-step process:

1. Let E be the original number on the right, 87.
2. Let F be the final number on the right, 59.
3. Calculate the difference G between the two: G = F - E = 59 - 87 = -28.
4. Let’s examine the numbers on the left side of the equation to find one that divides evenly into G: 8, 3, 7, 9. 7 divides evenly into -28.
5. Calculate J = G / H, here G is -28 and H is 7, so J = -28 / 7 = -4.
6. Verifying that J is an integer and H * J = G, we see that 7 * -4 = -28, which equals G (which is correct).
7. F